In [8]:
import cv2
import numpy as np
import face_recognition
import os
from datetime import datetime

In [9]:
path = 'images attendance/'
images1 = []
classNames1 = []
myList = os.listdir(path)
print(myList)
for cl in myList:
    curImg = cv2.imread(f'{path}/{cl}')
    images1.append(curImg)
    classNames1.append(os.path.splitext(cl)[0])
classNames = classNames1[1:]
images = images1[1:]
print(classNames)
#print(images)

['.ipynb_checkpoints', 'bill_gates.jpg', 'elon_musk.jpg', 'harshit.jpg', 'tony_stark.jpg']
['bill_gates', 'elon_musk', 'harshit', 'tony_stark']


In [10]:
def findEncodings(images):
    encodeList = []
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(img)[0]
        encodeList.append(encode)
    return encodeList

In [11]:
encodeListKnown = findEncodings(images)
print(len(encodeListKnown))

4


In [12]:
def markAttendance(name):
    with open('Attendance.csv','r+') as f:
        myDataList = f.readlines()
        nameList = []
        for line in myDataList:
            entry = line.split(',')
            nameList.append(entry[0])
        if name not in nameList:
            now = datetime.now()
            dtString = now.strftime('%H:%M:%S')
            f.writelines(f'\n{name},{dtString}')

In [13]:
#markAttendance('elon')
import sqlite3
sqlite3.connect('employ.db')

def get_info_from_db(text):
    # Connect to the SQLite database (or create it if it doesn't exist)
    conn = sqlite3.connect('employ.db')

    # Create a cursor object
    cur = conn.cursor()

    # Execute a SELECT query to get the row that matches the text
    cur.execute("SELECT * FROM employer WHERE Name = ?", (text,))

    # Fetch the first row from the results
    row = cur.fetchone()

    # Close the connection
    conn.close()

    # Return the row
    return row

In [14]:
cap = cv2.VideoCapture(0)
 
while True:
    success, img = cap.read()
#img = captureScreen()
    imgS = cv2.resize(img,(0,0),None,0.25,0.25)
    imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)
 
    facesCurFrame = face_recognition.face_locations(imgS)
    encodesCurFrame = face_recognition.face_encodings(imgS,facesCurFrame)
 
    for encodeFace,faceLoc in zip(encodesCurFrame,facesCurFrame):
        matches = face_recognition.compare_faces(encodeListKnown,encodeFace)
        faceDis = face_recognition.face_distance(encodeListKnown,encodeFace)
        #print(faceDis)
        matchIndex = np.argmin(faceDis)
 
        if matches[matchIndex]:
            name = classNames[matchIndex].upper()
            #print(name)
            y1,x2,y2,x1 = faceLoc
            y1, x2, y2, x1 = y1*4,x2*4,y2*4,x1*4
            cv2.rectangle(img,(x1,y1),(x2,y2),(0,255,0),2)
            cv2.rectangle(img,(x1,y2-35),(x2,y2),(0,255,0),cv2.FILLED)
            cv2.putText(img,name,(x1+6,y2-6),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),2)
            chiko = get_info_from_db(name)
            markAttendance(chiko)
 
    cv2.imshow('Webcam',img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture and destroy windows
cap.release()
cv2.destroyAllWindows()
